# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115833e+02     1.630674e+00
 * time: 0.604485034942627
     1     1.072321e+01     8.944018e-01
 * time: 2.4531030654907227
     2    -1.153103e+01     1.078476e+00
 * time: 2.939941167831421
     3    -3.402372e+01     7.920185e-01
 * time: 3.6629209518432617
     4    -4.747374e+01     5.367627e-01
 * time: 4.398113012313843
     5    -5.686378e+01     1.968141e-01
 * time: 5.106090068817139
     6    -5.986296e+01     1.038102e-01
 * time: 5.58432412147522
     7    -6.093503e+01     5.055322e-02
 * time: 6.076694965362549
     8    -6.130148e+01     6.428607e-02
 * time: 6.555086135864258
     9    -6.159840e+01     3.144936e-02
 * time: 7.033979177474976
    10    -6.179833e+01     2.586133e-02
 * time: 7.5141921043396
    11    -6.196796e+01     2.334085e-02
 * time: 8.000774145126343
    12    -6.203689e+01     1.937043e-02
 * time: 8.482371091842651
    13    -6.210919e+01     1.537489e-02
 * time: 8.961302042007446
    14  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557699
    AtomicNonlocal      14.8522663
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666456874
